In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [13]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os

fantom = 'https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange'

## Fantom

In [14]:
sample_transport=RequestsHTTPTransport(
    url=fantom,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

query = gql('''
query {
  factories(where: {liquidityUSD_gt:0}) {
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

665


In [15]:
response_list = []
id_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      name
                      id
                  }
                }
                '''
        response = client.execute(gql(querystring))
        for row in response['pairs']:
            response_list.append(row['name'])
            id_list.append(row['id'])    

df_fantom = pd.DataFrame.from_dict(list(zip(response_list,id_list)))
df_fantom[['Token 0', 'Token 1']] = df_fantom[0].str.split('-', expand = True, n = 1)
df_fantom

,0,1,Token 0,Token 1
0,FETH-FUSD,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,FETH,FUSD
1,USDC-BITB,0x01a20fe8a8c69250c7ac6814474293dfb7ccf337,USDC,BITB
2,WFTM-AQUT,0x0218bc593acfafbad1765dbede6d9bae94364eee,WFTM,AQUT
3,WFTM-DEFLATON,0x0281596a54d2876e827dc2eeb2102210f204decd,WFTM,DEFLATON
4,WFTM-3030,0x029c97f8019f01136536176bfffa60686e8478d8,WFTM,3030
...,...,...,...,...
385,WFTM-SAFEART,0xfc3a13d4764fc9bfa4cfc2bd8c075424fb225bbc,WFTM,SAFEART
386,WFTM-CORGI,0xfc93c4f5beb99aa7682437cd8ee2d2b821b35704,WFTM,CORGI
387,WFTM-SNX,0xfe69403cf2e22390c0d87ab05062f67d9084935b,WFTM,SNX
388,SAFEMOON-WFTM,0xfed2c6bc733b0f254c8009d5fcccb1c36e3381f5,SAFEMOON,WFTM


In [16]:
interesting_tokens = ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH', 'WBTC']
cream_borrowing_tokens = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI',\
'YFI',
'SNX',
'BAND',
'AAVE',
'COVER',
'CREAM',
'HEGIC']

interesting_tokens = ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH', 'WBTC']
combined_tokens = cream_borrowing_tokens + interesting_tokens


criteria = df_fantom['Token 0'].isin(combined_tokens) & \
           df_fantom['Token 1'].isin(combined_tokens)

df_fantom_filtered = df_fantom.loc[criteria,:]
df_fantom_filtered.columns = ['Pair', 'id', 'Token 0', 'Token 1']
df_fantom_filtered

,Pair,id,Token 0,Token 1
0,FETH-FUSD,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,FETH,FUSD
48,USDC-FUSD,0x19aea462d6d917a75b8971711d0af44f76e129d7,USDC,FUSD
54,ETH-FUSD,0x1d6101825d3412925e56c058559e6bfcc4661f1c,ETH,FUSD
107,WFTM-ETH,0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b,WFTM,ETH
161,DAI-FUSD,0x673c97c8040a615444d3b56f3f17075c30fa3f0a,DAI,FUSD
177,ETH-DAI,0x71c8bceece3daf9e27741d2cc1f03170f862555f,ETH,DAI
215,WFTM-FUSD,0x8623836f527350ec50691479674df0cd7773810c,WFTM,FUSD
222,fUSDT-FUSD,0x8a6eba896d750d8c04e815b74b3744d97bd0daad,fUSDT,FUSD
223,USDC-fUSDT,0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e,USDC,fUSDT
255,USDC-WFTM,0xa48869049e36f8bfe0cc5cf655632626988c0140,USDC,WFTM


In [17]:
df_fantom_filtered

,Pair,id,Token 0,Token 1
0,FETH-FUSD,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,FETH,FUSD
48,USDC-FUSD,0x19aea462d6d917a75b8971711d0af44f76e129d7,USDC,FUSD
54,ETH-FUSD,0x1d6101825d3412925e56c058559e6bfcc4661f1c,ETH,FUSD
107,WFTM-ETH,0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b,WFTM,ETH
161,DAI-FUSD,0x673c97c8040a615444d3b56f3f17075c30fa3f0a,DAI,FUSD
177,ETH-DAI,0x71c8bceece3daf9e27741d2cc1f03170f862555f,ETH,DAI
215,WFTM-FUSD,0x8623836f527350ec50691479674df0cd7773810c,WFTM,FUSD
222,fUSDT-FUSD,0x8a6eba896d750d8c04e815b74b3744d97bd0daad,fUSDT,FUSD
223,USDC-fUSDT,0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e,USDC,fUSDT
255,USDC-WFTM,0xa48869049e36f8bfe0cc5cf655632626988c0140,USDC,WFTM


In [18]:
df4 = pd.DataFrame([])
lenid = len(df_fantom_filtered['id'])

client = Client(transport=sample_transport)

for count, id in enumerate(df_fantom_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 7) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)
    
df4

 1 / 16 pair: 0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
 2 / 16 pair: 0x19aea462d6d917a75b8971711d0af44f76e129d7
 3 / 16 pair: 0x1d6101825d3412925e56c058559e6bfcc4661f1c
 4 / 16 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 5 / 16 pair: 0x673c97c8040a615444d3b56f3f17075c30fa3f0a
 6 / 16 pair: 0x71c8bceece3daf9e27741d2cc1f03170f862555f
 7 / 16 pair: 0x8623836f527350ec50691479674df0cd7773810c
 8 / 16 pair: 0x8a6eba896d750d8c04e815b74b3744d97bd0daad
 9 / 16 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
10 / 16 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
11 / 16 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
12 / 16 pair: 0xbeddbae6e24314a3336ef5e706a9a039395d6cb7
13 / 16 pair: 0xd0154be15563c100a90c9b4499f54e0f2f3098c6
14 / 16 pair: 0xd019dd7c760c6431797d6ed170bffb8faee11f99
15 / 16 pair: 0xd32f2eb49e91aa160946f3538564118388d6246a
16 / 16 pair: 0xe74385dae8fe6ebd0d55300300fe2e298324f95f


,date,reserveUSD,volumeUSD,id
0,1623542400,22.40272364584131170502567560046666,1.324605115142280281922330524863055,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
1,1623456000,22.64164439206274375845166157091055,0.8084026713955282324451786942704948,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
2,1623369600,21.14863403941647205976425724111638,0.585826291926230296895723054844739,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
3,1623196800,19.84067099777344981854330204678941,0.3095278451641449235812385722989809,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
4,1623024000,22.29946892111873375091650082412716,0.5574941993771236980628011677382221,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245
...,...,...,...,...
2,1619395200,0.1612020135153329656637412042383234,0.03745061554871135698186229890174748,0xe74385dae8fe6ebd0d55300300fe2e298324f95f
3,1619308800,0.1462956740855783324226335576144283,0.01463162137271587824271473162033248,0xe74385dae8fe6ebd0d55300300fe2e298324f95f
4,1619222400,0.1393870079011424141764067092342998,0.007736999994051390159474587546706083,0xe74385dae8fe6ebd0d55300300fe2e298324f95f
5,1619136000,0.1488656733709513392349476678818693,0.03237840340723737096289498401190672,0xe74385dae8fe6ebd0d55300300fe2e298324f95f


In [19]:
df5 = df4.merge(df_fantom_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s')
df5['fee'] = round(df5['volumeUSD'] * 0.003,6)
df5['1y APR for 100 invested'] = round((100/df5['reserveUSD']) * df5['fee'] * 365,3)
df5['Pair'] = '[Sushi Fantom] ' + df5['Pair']
df5.style.format({'reserveUSD': "{:0<4,.10f}"})

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,1y APR for 100 invested
0,2021-06-13 00:00:00,22.4027236458,1.324605,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.003974,6.475000
1,2021-06-12 00:00:00,22.6416443921,0.808403,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.002425,3.909000
2,2021-06-11 00:00:00,21.1486340394,0.585826,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.001757,3.032000
3,2021-06-09 00:00:00,19.8406709978,0.309528,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.000929,1.709000
4,2021-06-07 00:00:00,22.2994689211,0.557494,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.001672,2.737000
5,2021-06-06 00:00:00,20.8678840634,1.536602,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.004610,8.063000
6,2021-06-05 00:00:00,19.7875279313,0.711881,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,[Sushi Polygon] FETH-FUSD,FETH,FUSD,0.002136,3.940000
7,2021-06-19 00:00:00,1.1164893628,0.022831,0x19aea462d6d917a75b8971711d0af44f76e129d7,[Sushi Polygon] USDC-FUSD,USDC,FUSD,0.000068,2.223000
8,2021-06-18 00:00:00,1.1385323984,0.190853,0x19aea462d6d917a75b8971711d0af44f76e129d7,[Sushi Polygon] USDC-FUSD,USDC,FUSD,0.000573,18.370000
9,2021-06-17 00:00:00,1.1314163086,0.064031,0x19aea462d6d917a75b8971711d0af44f76e129d7,[Sushi Polygon] USDC-FUSD,USDC,FUSD,0.000192,6.194000


In [20]:
criteria1 = df5['reserveUSD'] > 1
criteria2 = df5['date'] == df5['date'].max()

latest_results = df5.loc[(criteria1 & criteria2), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)\
                    .drop(columns = ['id', 'Token 0', 'Token 1'])\
                    .reset_index(drop = True)[['date', 'Pair', '1y APR for 100 invested',  'reserveUSD', 'volumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'volumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["Pair"], **{'text-align': 'left'})

df_styled

,date,Pair,1y APR for 100 invested,reserveUSD,volumeUSD,fee
0,2021-06-19 00:00:00,[Sushi Polygon] USDC-WFTM,39.28,"196,154.04","70,362.05",211.09
1,2021-06-19 00:00:00,[Sushi Polygon] fUSDT-WFTM,10.47,"476,881.41","45,598.67",136.80
2,2021-06-19 00:00:00,[Sushi Polygon] WFTM-DAI,9.43,"458,470.93","39,493.99",118.48
3,2021-06-19 00:00:00,[Sushi Polygon] WFTM-FUSD,9.15,"1,030.52",86.13,0.26
4,2021-06-19 00:00:00,[Sushi Polygon] WFTM-ETH,5.07,"648,640.47","30,058.13",90.17
5,2021-06-19 00:00:00,[Sushi Polygon] USDC-fUSDT,3.06,"14,200.26",396.18,1.19
6,2021-06-19 00:00:00,[Sushi Polygon] fUSDT-FUSD,2.97,70.82,1.92,0.01
7,2021-06-19 00:00:00,[Sushi Polygon] USDC-DAI,2.60,287.74,6.82,0.02
8,2021-06-19 00:00:00,[Sushi Polygon] USDC-FUSD,2.22,1.12,0.02,0.00
9,2021-06-19 00:00:00,[Sushi Polygon] ETH-FUSD,1.10,121.62,1.22,0.00


In [21]:
dfi.export(df_styled, 'df_styled_sushi_fantom.png')
os.startfile('df_styled_sushi_fantom.png')